In [11]:
# create a function that return the general information of that compound e.g., number of each atom, MW, unsaturation

from rdkit import Chem
from rdkit.Chem import AllChem

def count_C(mol):
    return sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 6)
def count_O(mol):
    return sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 8)
def count_N(mol):
    return sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 7)
def count_P(mol):
    return sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 15)
def count_S(mol):
    return sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 16)
def count_X(mol):
    return sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 9 or atom.GetAtomicNum() == 17 or atom.GetAtomicNum() == 35 or atom.GetAtomicNum() == 53)
def count_H(mol):
    H = 0
    for i in range(mol.GetNumAtoms()):
        H += mol.GetAtomWithIdx(i).GetTotalNumHs(includeNeighbors=True)
    return H

def DoU(mol):
    C = count_C(mol)
    H = count_H(mol)
    O = count_O(mol)
    N = count_N(mol)
    P = count_P(mol)
    S = count_S(mol)
    X = count_X(mol)
    return float((2*C+2+N+P-X-H)/2)

from rdkit.Chem.Descriptors import MolWt

def cpd_inform(SMILES):
    
    """A function for getting compound information from SMILES string
    it received a SMILES string and return a dictionary of information consisted of number of C, H, O , N, P, S, X, Degree of Unsaturation and Molecular Weight"""
    info = {}
    mol = Chem.MolFromSmiles(SMILES)
    info['n_C'] = float(count_C(mol))
    info['n_H'] = float(count_H(mol))
    info['n_O'] = float(count_O(mol))
    info['n_N'] = float(count_N(mol))
    info['n_P'] = float(count_P(mol))
    info['n_S'] = float(count_S(mol))
    info['n_X'] = float(count_X(mol))
    info['DoU'] = DoU(mol)
    info['MW'] = MolWt(mol)
    
    return info

In [12]:
from rdkit import Chem
from rdkit.Chem import AllChem
from cpd_inform import cpd_inform
def test_cpd_inform():
    rapamycin = 'C[C@@H]1CC[C@H]2C[C@@H](/C(=C/C=C/C=C/[C@H](C[C@H](C(=O)[C@@H]([C@@H](/C(=C/[C@H](C(=O)C[C@H](OC(=O)[C@@H]3CCCCN3C(=O)C(=O)[C@@]1(O2)O)[C@H](C)C[C@@H]4CC[C@H]([C@@H](C4)OC)O)C)/C)O)OC)C)C)/C)OC'
    test = cpd_inform(rapamycin)
    
    assert test['n_C'] == 51, "Carbon count is incorrect"
    assert test['n_H'] == 79, "Hydrogen count is incorrect"
    assert type(test['MW']) == type(1.0), "TypeError: Molecular Weight should be float"
    
    return 'Test pass, yayyyyyy'

In [13]:
rapamycin = 'C[C@@H]1CC[C@H]2C[C@@H](/C(=C/C=C/C=C/[C@H](C[C@H](C(=O)[C@@H]([C@@H](/C(=C/[C@H](C(=O)C[C@H](OC(=O)[C@@H]3CCCCN3C(=O)C(=O)[C@@]1(O2)O)[C@H](C)C[C@@H]4CC[C@H]([C@@H](C4)OC)O)C)/C)O)OC)C)C)/C)OC'
test = cpd_inform(rapamycin)
test

{'n_C': 51.0,
 'n_H': 79.0,
 'n_O': 13.0,
 'n_N': 1.0,
 'n_P': 0.0,
 'n_S': 0.0,
 'n_X': 0.0,
 'DoU': 13.0,
 'MW': 914.1870000000001}

In [14]:
test['n_C']

51.0

In [15]:
test_cpd_inform()

'Test pass, yayyyyyy'

In [37]:
# Create a function that create a new column of chemical information

import pandas as pd
from cpd_inform import cpd_inform

def create_cpd_info(df, col_name='SMILES'):
    
    """Receive a DataFrame and return a dataframe with additional columns named n_C, n_H, ..., DoU, and MW"""
    
    df['n_C'] = 'Empty'
    df['n_H'] = 'Empty'
    df['n_O'] = 'Empty'
    df['n_N'] = 'Empty'
    df['n_P'] = 'Empty'
    df['n_S'] = 'Empty'
    df['n_X'] = 'Empty'
    df['DoU'] = 'Empty'
    df['MW'] = 'Empty'
    
    for index in range(df.shape[0]):
        mol = df[col_name][index]
        info = cpd_inform(mol)
        df['n_C'][index] = info['n_C']
        df['n_H'][index] = info['n_H']
        df['n_O'][index] = info['n_O']
        df['n_N'][index] = info['n_N']
        df['n_P'][index] = info['n_P']
        df['n_S'][index] = info['n_S']
        df['n_X'][index] = info['n_X']
        df['DoU'][index] = info['DoU']
        df['MW'][index] = info['MW']
        
    return df

In [38]:
create_cpd_info(df_master, 'SMILES')

,SMILES,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
0,C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O,6,12,6,0,0,0,0,1,180.156
1,C([C@@H]1[C@@H]([C@@H]([C@H]([C@H](O1)O)O)O)O)O,6,12,6,0,0,0,0,1,180.156
2,C([C@H]([C@H]([C@@H](C(=O)CO)O)O)O)O,6,12,6,0,0,0,0,1,180.156
3,C[C@@H]1CC[C@H]2C[C@@H](/C(=C/C=C/C=C/[C@H](C[...,51,79,13,1,0,0,0,13,914.187


In [40]:
create_cpd_info(df_master)

,SMILES,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
0,C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O,6,12,6,0,0,0,0,1,180.156
1,C([C@@H]1[C@@H]([C@@H]([C@H]([C@H](O1)O)O)O)O)O,6,12,6,0,0,0,0,1,180.156
2,C([C@H]([C@H]([C@@H](C(=O)CO)O)O)O)O,6,12,6,0,0,0,0,1,180.156
3,C[C@@H]1CC[C@H]2C[C@@H](/C(=C/C=C/C=C/[C@H](C[...,51,79,13,1,0,0,0,13,914.187


In [43]:
import pandas as pd
from create_cpd_info import create_cpd_info

def test_create_cpd_info():
    
    """A unit test for create compound info"""
    
    df_master = pd.DataFrame(['C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O',
         'C([C@@H]1[C@@H]([C@@H]([C@H]([C@H](O1)O)O)O)O)O',
         'C([C@H]([C@H]([C@@H](C(=O)CO)O)O)O)O', 
         'C[C@@H]1CC[C@H]2C[C@@H](/C(=C/C=C/C=C/[C@H](C[C@H](C(=O)[C@@H]([C@@H](/C(=C/[C@H](C(=O)C[C@H](OC(=O)[C@@H]3CCCCN3C(=O)C(=O)[C@@]1(O2)O)[C@H](C)C[C@@H]4CC[C@H]([C@@H](C4)OC)O)C)/C)O)OC)C)C)/C)OC'] , columns=['SMILES'])
    test = create_cpd_info(df_master)
    
    assert test['n_C'][0] == 6, "ValueError: Carbon count is incorrect"
    assert test['DoU'][3] == 13, "ValueError: Degree of Unsaturation in inaccurate"
    assert type(test['MW'][2]) == type(5.0), "TypeError: MW should be float"
    assert type(test['n_H'][3]) == type(1.0), "TypeError: All data should be float"
    
    return 'Test pass, you can use it to create compound info columns'

In [44]:
test_create_cpd_info()

'Test pass, you can use it to create compound info columns'

In [25]:
df_master['n_C'] = '0.0'
df_master

,SMILES,n_C
0,C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O,0.0
1,C([C@@H]1[C@@H]([C@@H]([C@H]([C@H](O1)O)O)O)O)O,0.0
2,C([C@H]([C@H]([C@@H](C(=O)CO)O)O)O)O,0.0
3,C[C@@H]1CC[C@H]2C[C@@H](/C(=C/C=C/C=C/[C@H](C[...,0.0
